In [ ]:
%tensorflow_version 2.x
import tensorflow as tf

# Check that GPU is available: cf. https://colab.research.google.com/notebooks/gpu.ipynb
assert(tf.config.list_physical_devices('GPU'))

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False) # Start with XLA disabled.
!pip install -U efficientnet
import efficientnet.keras as efn 
import os
import numpy as np
import cv2
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from keras.models import load_model
from tensorflow.keras.models import Model
import keras
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Sequential
from sklearn.metrics import confusion_matrix, classification_report
import itertools

from sklearn.preprocessing import OneHotEncoder

tf.config.optimizer.set_jit(False) # Start with XLA disabled.

%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
batch_size = 64  # The default batch size of keras.
num_classes = 100  # Number of class for the dataset
epochs = 150
data_augmentation = True

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

169009152/169001437 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = tf.keras.utils.to_categorical(y_train, num_classes = 100)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=100)

In [ ]:
# !pip install -U efficientnet

Requirement already up-to-date: efficientnet in /usr/local/lib/python3.6/dist-packages (1.1.0)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(weights = 'imagenet', include_top=False, input_shape=(256,256,3))

219062272/219055592 [==============================] - 4s 0us/step


In [ ]:
# base_model = tf.keras.applications.
# (weights = 'imagenet', include_top=False, input_shape=(128,128,3)) #

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.UpSampling2D())
model.add(tf.keras.layers.UpSampling2D())
model.add(tf.keras.layers.UpSampling2D())
# model.add(tf.keras.layers.UpSampling2D())
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(4096, activation = 'elu'))
model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(1024, activation = 'elu'))
model.add(tf.keras.layers.Dense(100, activation = 'softmax'))

model.build(input_shape = [None,32,32,3])

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.01, momentum = 0.9)

# Let's train the model using RMSproptf.
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_15 (UpSampling multiple                  0         
_________________________________________________________________
up_sampling2d_16 (UpSampling multiple                  0         
_________________________________________________________________
up_sampling2d_17 (UpSampling multiple                  0         
_________________________________________________________________
inception_resnet_v2 (Model)  (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_5 ( multiple                  0         
_________________________________________________________________
dense_15 (Dense)             multiple                  6295552   
_________________________________________________________________
dropout_6 (Dropout)          multiple                 

In [ ]:
# parameters for callback functions
es_patience = 10
rlrop_patience = 5
decay_rate = 0.5

In [ ]:
history = None  # For recording the history of trainning process.
if not data_augmentation:
    print('Not using data augmentation.')
    my_callbacks = [
      keras.callbacks.EarlyStopping(patience=25),
      keras.callbacks.ModelCheckpoint(filepath='best_model.h5',
                                         save_best_only=True, monitor='val_loss', mode='min'),
    ]
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=my_callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.12,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.12,
        shear_range=0.,  # set range for random shear
        zoom_range=0.17,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    my_callbacks = [
      tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/Colab Notebooks/best_model12.h5',
                                        save_best_only=True, monitor='val_accuracy', mode='max'),
      tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', patience = 15, restore_best_weights = True, verbose = 1),
      tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', mode = 'min', patience = 4, factor = decay_rate, min_lr = 1e-6, verbose = 1)
    ]
    # Fit the model on the batches generated by datagen.flow().

Using real-time data augmentation.


In [ ]:
model.fit(datagen.flow(x_train, y_train,
                        batch_size=batch_size),
                        shuffle = True,
                        steps_per_epoch=len(x_train) / 256,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=8,
                        callbacks=my_callbacks)

Epoch 1/150
196/195 [==============================] - 816s 4s/step - loss: 0.7625 - accuracy: 0.7772 - val_loss: 0.8540 - val_accuracy: 0.7546 - lr: 0.0100
Epoch 2/150
196/195 [==============================] - 808s 4s/step - loss: 0.7258 - accuracy: 0.7879 - val_loss: 0.8777 - val_accuracy: 0.7535 - lr: 0.0100
Epoch 3/150
196/195 [==============================] - 817s 4s/step - loss: 0.6531 - accuracy: 0.8092 - val_loss: 0.7971 - val_accuracy: 0.7744 - lr: 0.0100
Epoch 4/150
196/195 [==============================] - 808s 4s/step - loss: 0.5977 - accuracy: 0.8172 - val_loss: 0.8600 - val_accuracy: 0.7692 - lr: 0.0100
Epoch 5/150
196/195 [==============================] - 809s 4s/step - loss: 0.5823 - accuracy: 0.8279 - val_loss: 0.9022 - val_accuracy: 0.7687 - lr: 0.0100
Epoch 6/150
196/195 [==============================] - 817s 4s/step - loss: 0.5573 - accuracy: 0.8312 - val_loss: 0.7659 - val_accuracy: 0.7845 - lr: 0.0100
Epoch 7/150
196/195 [==============================] - 819